In [6]:
import ipywidgets as widgets
from ipywidgets import interact, HBox, VBox
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pyroomacoustics as pra
import numpy as np
import random
# from matplotlib.widgets import RectangleSelector

In [7]:
def plot_room_and_positions(room_x, room_y, room_z, mic_x, mic_y, mic_z, src_x, src_y, src_z):
    mic_x, mic_y, mic_z = mic_x * room_x, mic_y * room_y, mic_z * room_z
    src_x, src_y, src_z = src_x * room_x, src_y * room_y, src_z * room_z
    fig = plt.figure(figsize=(10, 7))
    # Plot the room and positions
    ax1 = fig.add_subplot(121, projection='3d')
    # Room dimensions
    ax1.plot([0, room_x, room_x, 0, 0], [0, 0, room_y, room_y, 0], [0, 0, 0, 0, 0], color='b')  # Bottom face
    ax1.plot([0, room_x, room_x, 0, 0], [0, 0, room_y, room_y, 0], [room_z, room_z, room_z, room_z, room_z], color='b')  # Top face
    ax1.plot([0, 0], [0, 0], [0, room_z], color='b')  # 4 vertical lines
    ax1.plot([room_x, room_x], [0, 0], [0, room_z], color='b')
    ax1.plot([room_x, room_x], [room_y, room_y], [0, room_z], color='b')
    ax1.plot([0, 0], [room_y, room_y], [0, room_z], color='b')
    # Microphone position
    ax1.scatter(mic_x, mic_y, mic_z, c='r', label='Mic')
    # Source position
    ax1.scatter(src_x, src_y, src_z, c='g', label='Source')
    ax1.set_xlabel('X')
    ax1.set_ylabel('Y')
    ax1.set_zlabel('Z')
    ax1.set_title('Room and Positions')
    ax1.legend()
    ax1.set_xlim([0, 10])
    ax1.set_ylim([0, 10])
    ax1.set_zlim([0, 10])

    # Generate and plot the RIR
    room = pra.ShoeBox([room_x, room_y, room_z], absorption=0.3, max_order=15)
    room.add_source([src_x, src_y, src_z])
    mic = pra.MicrophoneArray(np.array([[mic_x, mic_y, mic_z]]).T, fs=room.fs)
    room.add_microphone_array(mic)
    room.compute_rir()
    rir = room.rir[0][0]
    ax2 = fig.add_subplot(122)

    mic_position = np.array([mic_x, mic_y, mic_z])
    src_position = np.array([src_x, src_y, src_z])
    # Compute Euclidean distance
    distance = np.linalg.norm(mic_position - src_position)
    # Compute time delay
    time_delay = distance / 343.0  # Speed of sound in air at room temperature
    # Compute samples delay
    samples_delay = int(np.round(time_delay * room.fs))
    ax2.plot(np.arange(len(rir)), abs(rir))
    ax2.set_xlim(0, 2000)
    ax2.set_ylim(-0.05, 1)
    ax2.set_title('Room Impulse Response')
    plt.tight_layout()
    plt.show()

# Define sliders
room_x_slider = widgets.FloatSlider(value=5.3, min=1, max=20, step=0.1, description='Room X:')
room_y_slider = widgets.FloatSlider(value=4.2, min=1, max=20, step=0.1, description='Room Y:')
room_z_slider = widgets.FloatSlider(value=3.1, min=1, max=100, step=0.1, description='Room Z:')
mic_x_slider = widgets.FloatSlider(value=random.random(), min=0, max=1.0, step=0.01, description='Mic X:')
mic_y_slider = widgets.FloatSlider(value=random.random(), min=0, max=1.0, step=0.01, description='Mic Y:')
mic_z_slider = widgets.FloatSlider(value=0.5, min=0, max=1.0, step=0.01, description='Mic Z:')
src_x_slider = widgets.FloatSlider(value=random.random(), min=0, max=1.0, step=0.01, description='Src X:')
src_y_slider = widgets.FloatSlider(value=random.random(), min=0, max=1.0, step=0.01, description='Src Y:')
src_z_slider = widgets.FloatSlider(value=0.5, min=0, max=1.0, step=0.01, description='Src Z:')

# Arrange sliders in a vertical box
slider_box = VBox([room_x_slider, room_y_slider, room_z_slider, mic_x_slider, mic_y_slider, mic_z_slider, src_x_slider, src_y_slider, src_z_slider])

# Create interactive plot
output = widgets.interactive_output(plot_room_and_positions, {
    'room_x': room_x_slider,
    'room_y': room_y_slider,
    'room_z': room_z_slider,
    'mic_x': mic_x_slider,
    'mic_y': mic_y_slider,
    'mic_z': mic_z_slider,
    'src_x': src_x_slider,
    'src_y': src_y_slider,
    'src_z': src_z_slider,
})

# Display sliders and plot side by side
display(HBox([slider_box, output]))